<a href="https://colab.research.google.com/github/jspumn/jspumn.github.io/blob/main/ASF_Asia_Timeline_(as_of_Nov_2025).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This script generates an interactive, zoomable timeline plot using Python's Plotly library.
# You can paste this entire code block into a Google Colab notebook cell and run it.

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from datetime import datetime, timedelta

# --- 1. Data Preparation ---

# Data from the timeline table (country and first outbreak date)
data = {
    'Country': [
        'China', 'Mongolia', 'Cambodia', 'Hong Kong', 'Bangladesh', 'North Korea',
        'Laos', 'Philippines', 'Myanmar', 'Indonesia', 'East Timor', 'South Korea',
        'India', 'Vietnam', 'Malaysia', 'Bhutan', 'Thailand', 'Nepal',
        'Singapore', 'Sri Lanka', 'Taiwan'
    ],
    'Date_Str': [
        '2018-08-01', '2019-01-09', '2019-03-22', '2019-05-02', '2019-05-13',
        '2019-05-23', '2019-06-02', '2019-07-25', '2019-08-01', '2019-09-04',
        '2019-09-09', '2019-09-16', '2020-01-26', '2020-02-02', '2021-02-08',
        '2021-05-06', '2021-11-22', '2022-03-03', '2023-01-05', '2024-10-25',
        '2025-10-21'
    ]
}

df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date_Str'])
df = df.sort_values(by='Date').reset_index(drop=True)

# Create an alternating vertical position (y_pos) for labels for visual separation
# Also create a clean date format for display
df['Label_Date'] = df['Date'].dt.strftime('%b %Y')
df['y_pos'] = np.where(df.index % 2 == 0, 1.0, -1.0)
df['Label'] = df['Country'] + ' (' + df['Label_Date'] + ')'

# --- 2. Plotly Visualization ---

# Define the primary color palette
timeline_color = '#E54B4B'
line_color = '#333333'
text_color = '#1F1F1F'
background_color = '#FAFAFA'

# 1. Create the base figure
fig = go.Figure()

# 2. Add the main horizontal timeline line
fig.add_trace(go.Scatter(
    x=df['Date'],
    y=[0] * len(df),
    mode='lines',
    line=dict(color=line_color, width=2),
    hoverinfo='none',
    showlegend=False
))

# 3. Add the event markers (dots)
fig.add_trace(go.Scatter(
    x=df['Date'],
    y=[0] * len(df),
    mode='markers',
    marker=dict(
        color=timeline_color,
        size=10,
        line=dict(width=1.5, color='white')
    ),
    # Custom hover text
    hovertext=df.apply(lambda row: f"<b>{row['Country']}</b><br>First Outbreak: {row['Label_Date']}", axis=1),
    hoverinfo='text',
    showlegend=False
))

# 4. Add the vertical lines and labels
for index, row in df.iterrows():
    # Vertical connecting line
    fig.add_trace(go.Scatter(
        x=[row['Date'], row['Date']],
        y=[0.05 * np.sign(row['y_pos']), row['y_pos'] * 1.5], # Start slightly above/below 0
        mode='lines',
        line=dict(color='gray', width=0.5, dash='dot'),
        hoverinfo='none',
        showlegend=False
    ))

    # Text label for country and date
    fig.add_annotation(
        x=row['Date'],
        y=row['y_pos'] * 1.6,
        text=row['Label'],
        showarrow=False,
        font=dict(color=text_color, size=10, family="Arial"),
        align="center",
        valign="middle",
        # Set alignment based on position
        yanchor='top' if row['y_pos'] < 0 else 'bottom',
        yshift=-10 if row['y_pos'] < 0 else 10 # Slight shift for clearance
    )

# --- 3. Layout Configuration for Interactivity and Aesthetics ---

# Generate annual lines for visual aid
min_year = df['Date'].min().year
max_year = df['Date'].max().year
year_breaks = [datetime(y, 1, 1) for y in range(min_year, max_year + 2)]
year_labels = [str(y) for y in range(min_year, max_year + 2)]

# Add vertical year markers
shapes = []
annotations = []
for date in year_breaks:
    shapes.append(dict(
        type='line',
        xref='x', yref='y',
        x0=date, x1=date,
        y0=-2.0, y1=2.0,
        line=dict(color="#CCCCCC", width=0.5, dash="dash")
    ))
    annotations.append(dict(
        x=date, y=-2.5, text=date.strftime("%Y"),
        showarrow=False, font=dict(size=12, color='#333333', family="Arial"),
        yanchor='top'
    ))

fig.update_layout(
    title={
        'text': "<b>Chronological Timeline of First Outbreaks in Asian Countries (2018–2025)</b>",
        'y': 0.95, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top'
    },
    # Set the overall background color
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,

    # Configure X-axis (Timeline)
    xaxis=dict(
        showgrid=False,
        zeroline=False,
        tickformat='%b',  # Show month name
        dtick='M3',       # Major ticks every 3 months
        # Set a clean range around the data
        range=[df['Date'].min() - timedelta(days=90), df['Date'].max() + timedelta(days=90)],
        title='',
        showline=False
    ),

    # Configure Y-axis (Hidden)
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showticklabels=False,
        title='',
        range=[-3.5, 3.5] # Ensure room for the year labels
    ),

    # Add the year markers as shapes and annotations
    shapes=shapes,
    annotations=annotations,

    # Other general settings
    height=600,
    showlegend=False,
    hovermode='x unified' # Optimized hover behavior for timelines
)

# Display the interactive plot
fig.show()